In [1]:
import numpy as np
from pygel3d import graph
from pygel3d import hmesh, gl_display as gl
from pygel3d import jupyter_display as jd
import pygel3d as py3d
from plyfile import PlyData, PlyElement

In [2]:
import bpy

In [3]:
points = np.load('../data/points.npy')
edges = np.load('../data/edge_array.npy')
radii = np.load('../data/edge_radius.npy')

print(points.shape)
print(edges.shape)
print(radii.shape)

with open('skeleton.obj', 'w') as f:
    # f.write(f'g default\n')
    for point in points:
        f.write(f'n {point[0]} {point[1]} {point[2]}\n')
        
    # for edge in edges:
    #     f.write(f'l {edge[0]+1} {edge[1]+1}\n')

    for i, edge in enumerate(edges):
        radius = radii[i]
        # f.write(f'c {edge[0]} {edge[1]} {radius}\n')
        f.write(f'c {edge[0]} {edge[1]}\n')
        
    f.close()

(50177, 3)
(50176, 2)
(50176,)


In [4]:
# tri_mesh = trimesh.Trimesh()
# tri_mesh.vertices = points
# edges_with_radii = []
# for i, edge in enumerate(edges):
#     vertex1, vertex2 = edge
#     radius = radii[i]
#     edges_with_radii.append([vertex1, vertex2, radius])

# tri_mesh.lines = edges_with_radii

# tri_graph = tri_mesh.export(file_type='obj')
# tri_mesh.show()

# print(edges.max())


In [5]:
def edge_radius_to_vertex_radius(edges, edge_radius):
    vertex_radius = np.zeros((edges.max()+1,))
    for i, edge in enumerate(edges):
        vertex_radius[edge[0]] = max(edge_radius[i], vertex_radius[edge[0]])     
        vertex_radius[edge[1]] = max(edge_radius[i], vertex_radius[edge[1]])
    return vertex_radius


def convert_custom_obj_to_standard(input_file, output_file):
    with open(input_file, 'r') as input_obj:
        lines = input_obj.readlines()

    converted_lines = []

    for line in lines:
        if line.startswith('n '):  # Change 'n' to 'v' for vertex
            parts = line.strip().split()
            converted_line = f"v {' '.join(parts[1:])}\n"
            converted_lines.append(converted_line)
        elif line.startswith('c '):  # Change 'c' to 'l' for edges
            parts = line.strip().split()
            converted_line = f"l {' '.join(parts[1:])}\n"
            converted_lines.append(converted_line)

    # Reindex the vertex indices to start from 0
    vertex_index_offset = 0
    for line in converted_lines:
        if line.startswith('l '):
            parts = line.strip().split()
            converted_line = f"l {' '}"
            for v in parts[1:]:
                converted_line += f"{int(v)+1} "
            converted_line += '\n'
            converted_lines[vertex_index_offset] = converted_line
        vertex_index_offset += 1

    with open(output_file, 'w') as output_obj:
        output_obj.writelines(converted_lines)

In [6]:
vertex_radius = edge_radius_to_vertex_radius(edges, radii)/10
np.save('vertex_radius.npy', vertex_radius)


In [7]:
# mesh = bpy.data.meshes.new('skeleton')
# mesh.from_pydata(points, edges, [])
# mesh.update()
# skeleton_object = bpy.data.objects.new('skeleton', mesh)
# skeleton_collection = bpy.data.collections.new('skeleton')
# bpy.context.scene.collection.children.link(skeleton_collection)
# skeleton_collection.objects.link(skeleton_object)



# if 'radii' not in mesh.data.attributes:
#     mesh.data.attributes.new('radii',0,0)
# attibute = mesh.data.attributes['radii']
# attibute.data.foreach_set('value', vertex_radius)

In [8]:
# with open('skeleton_vradii.obj', 'w') as f:
#     for i, radius in enumerate(vertex_radius):
#         f.write(f'v {points[i][0]} {points[i][1]} {points[i][2]}\n')
#     for i, edge in enumerate(edges):
#         f.write(f'l {edge[0]+1} {edge[1]+1}\n')
#     f.close()

In [9]:

g = graph.load("./skeleton.obj")
print(g)
print(vertex_radius)
print(points)


[31.44276726 31.44276726 17.33434249 ...  1.44948413  1.27329233
  1.26522435]
[[588.         217.         650.        ]
 [585.184      206.246      641.041     ]
 [437.314      266.05       394.903     ]
 ...
 [182.97035217  88.3454361  228.9609375 ]
 [303.37478961 471.17907713 416.99790026]
 [ 69.09300419 397.80529598 561.17693889]]


In [10]:
# skeleton = graph.LS_skeleton(g)

In [11]:
# graph.save('../data/skeleton.obj', skeleton)
# convert_custom_obj_to_standard('../data/skeleton.obj', '../data/skeleton_standard.obj')

In [12]:
# generate point cloud

import open3d as o3d
import open3d.core as o3c

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
o3d.io.write_point_cloud("../data/pcd.ply", pcd)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


True

In [35]:
pcd_load = o3d.io.read_point_cloud("../data/pcd.ply")
pcd_load.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(len(points), 3)))
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd_load, voxel_size=20)

points_load = np.asarray(pcd_load.points)

# o3d.visualization.draw_geometries([pcd_load])
o3d.io.write_voxel_grid("../data/voxel_grid.ply", voxel_grid)


True

In [36]:
voxels = voxel_grid.get_voxels()
indices = np.stack(list(vx.grid_index for vx in voxels))
print(indices)


[[22 19 19]
 [22 19 12]
 [22 16  9]
 ...
 [16 19 41]
 [16 19 18]
 [16 20 21]]


In [37]:
def voxel_grid_from_idx(indices):
    voxel_grid = np.zeros((indices.max(axis=0)+1))
    for index in indices:
        voxel_grid[tuple(index)] = 1
    return voxel_grid

voxel_grid = voxel_grid_from_idx(indices)

In [38]:
iso_mesh = 0
iso_mesh = hmesh.volumetric_isocontour(voxel_grid,high_is_inside=True,make_triangles=False)

In [39]:
hmesh.save('../data/iso_mesh.obj', iso_mesh)

In [40]:
viewer = gl.Viewer()
viewer.display(iso_mesh)
del viewer

In [10]:
# with open('../data/skeleton.obj', 'w') as f:
#     # f.write(f'g default\n')
#     for node in skeleton.positions():
#         f.write(f'n {node[0]} {node[1]} {node[2]}\n')
        
#     # for edge in edges:
#     #     f.write(f'l {edge[0]+1} {edge[1]+1}\n')

#     for i, edge in enumerate(edges):
#         radius = radii[i]
#         # f.write(f'c {edge[0]} {edge[1]} {radius}\n')
#         f.write(f'c {edge[0]} {edge[1]}\n')
        
#     f.close()

True

In [ ]:
# build volume


In [36]:
# m = graph.to_mesh_cyl(g,1)
FEQ_mesh = hmesh.skeleton_to_feq(g,vertex_radius)
hmesh.save("feq_mesh_true.obj",FEQ_mesh)

In [39]:
FEQ_mesh_t1 = hmesh.skeleton_to_feq(g,vertex_radius*10)


In [40]:
hmesh.save("feq_mesh_true1.obj",FEQ_mesh_t1)

In [ ]:
FEQ_mesh_001 = hmesh.skeleton_to_feq(g, radii/100)
FEQ_mesh_01 = hmesh.skeleton_to_feq(g, radii/10)
FEQ_mesh_1 = hmesh.skeleton_to_feq(g, radii)
# hmesh.loop_split(m)
# hmesh.loop_smooth(m)

In [21]:
hmesh.save("feq_mesh_scale0.01.obj", FEQ_mesh_001)
hmesh.save("feq_mesh_scale0.1.obj", FEQ_mesh_01)
hmesh.save("feq_mesh_scale1.obj", FEQ_mesh_1)

In [5]:
viewer = gl.Viewer()
viewer.display(m)


In [6]:
del viewer

In [7]:
hmesh.save('mesh_example.obj',m)